In [7]:
'''
Take all of the results and make a big table

Also make the timing plot
'''

import csv

from subprocess import Popen, PIPE
process = Popen(['RS', 'scripts/latencies.R'], stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()

from subprocess import Popen, PIPE
process = Popen(['RS', 'scripts/rollup_times.R'], stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()

fn = "bottom_up_clean/all_times_rollup.csv"

method_to_full_name = {"random": "make_decision_random", "additive":"make_decision_lr",
                       "ilp": "ilp", "ablated":"only_locals",
                      "additive_nn": "make_decision_nn"}

method2time_mu = {}

with open(fn, "r") as inf:
    reader = csv.reader(inf)
    head = next(reader)
    for o in reader:
        method, mean = o
        method2time_mu[method_to_full_name[method]] = float(mean) * 1000 # times 1000 for ms


import csv
from collections import defaultdict

results_fn = "bottom_up_clean/results.csv"

results = []

method2slormu = defaultdict(float)
method2slor_std = defaultdict(float)
method2f1 = defaultdict(float)

print_method = {}
print_method["make_decision_lr"] = "\\textsc{vertex addition}"
print_method["only_locals"] = "\\textsc{ablated} {\\small (edge only) }"
print_method["make_decision_random"] = "\\textsc{random} {\\small (lower bound) }"
print_method["ilp"] = "\\textsc{ilp}"
print_method["make_decision_nn"] = "\\textsc{nn}"


with open(results_fn, "r") as inf:
    reader = csv.reader(inf)
    header = next(reader)
    for ln in reader:
        f1, slor_mu, slor_std, method = ln
        method2f1[method] = float(f1)
        method2slormu[method] = float(slor_mu)
        method2slor_std[method] = float(slor_std)


def get_format(sigfig):
    return "{:." + str(sigfig) + "f}"

def todec(float_, sigfig=3):
    dec = get_format(sigfig).format(float_)
    small = "{\small " + dec + "}"
    return small


def todec_bold(float_,sigfig=3):
    dec = get_format(sigfig).format(float_)
    small = "\\textbf{\small " + dec + "}"
    return small


for method in ['make_decision_random', 'ilp', 'only_locals', 'make_decision_lr', "make_decision_nn"]:
    if method != "make_decision_lr":
        print("&".join([print_method[method],
              todec(method2f1[method]),
              todec(method2slormu[method]),
              todec(method2time_mu[method], 1),
              ]) + "\\\\")
    else:
        print("&".join(["\\textbf{" + print_method[method] + "}",
              todec_bold(method2f1[method]),
              todec_bold(method2slormu[method]),
              todec_bold(method2time_mu[method], 1),
              ]) + "\\\\")

\textsc{random} {\small (lower bound) }&{\small 0.653}&{\small 0.377}&{\small 0.5}\\
\textsc{ilp}&{\small 0.852}&{\small 0.756}&{\small 44.0}\\
\textsc{ablated} {\small (edge only) }&{\small 0.827}&{\small 0.669}&{\small 3.7}\\
\textbf{\textsc{vertex addition}}&\textbf{\small 0.881}&\textbf{\small 0.745}&\textbf{\small 4.1}\\
\textsc{nn}&{\small 0.873}&{\small 0.729}&{\small 2929.1}\\
